# RadTract example pipeline

This notebook will show you how to use RadTract to calculate radiomics tractometry features. You can download the sample data used in this example here: https://doi.org/10.5281/zenodo.4139626.

To run RadTract on a single subject, you need at least the following items:

1. A tractogram, i.e., a collection of streamlines, for the tract you are interested in. 
2. The image you want to calculate features from, for example a fractional anisotropy (FA) map.

The code below is the tl;dr version of how to use these items to calculate your tract-specific features. Keep in mind to read the longer example below before using RadTract for a multi-subject study!

In [17]:
!radtract_parcellate --streamlines /home/neher/Downloads/RadTract_Example/CST_right.trk --reference /home/neher/Downloads/RadTract_Example/fa.nii.gz --output /home/neher/Downloads/RadTract_Example/CST_right_parcellation.nii.gz
!radtract_features --parcellation /home/neher/Downloads/RadTract_Example/CST_right_parcellation.nii.gz --map /home/neher/Downloads/RadTract_Example/fa.nii.gz --output /home/neher/Downloads/RadTract_Example/CST_right_features.csv

Creating binary envelope from reference image
Calculating bundle envelope
done
Input number of fibers: 14026
Estimating number of possible parcels for on average 5 traversed voxels per parcel.
Number of estimated parcels 10
Creating local reference centroid
Reorienting streamlines...
Reducing input bundle
Reduced number of fibers: 522
Fitting parcellation model to 5220 points
Predicting hyperplane-parcellation for 2973 voxels
Finished hyperplane-based parcellation
Saving hyperplane-based parcellation to /home/neher/Downloads/RadTract_Example/CST_right_parcellation.nii.gz
Pyradiomics settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 0.5, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'correctMask': True, 'binWidth': 0.02}
Enabled image types: {'Or

The above example uses default parameters and might not be optimal for your usecase. For example, the automatically estimated number of parcels might vary between subjects, thus rendering them incompatible for a future joint statistical analysis or machine learning experiment. To avoid this, it is possible to manually set the number of parcel (`--num_parcels`).

RadTract further offers multiple types of parcellations, namely hyperplane (default), centerline and static.




In [18]:
!radtract_parcellate

usage: radtract_parcellate [-h] [--streamlines STREAMLINES]
                           [--envelope ENVELOPE] [--reference REFERENCE]
                           [--start START] [--num_parcels NUM_PARCELS]
                           [--type TYPE]
                           [--save_intermediate_files SAVE_INTERMEDIATE_FILES]
                           [--streamline_space STREAMLINE_SPACE]
                           [--output OUTPUT]

RadTract Tract Parcellation

options:
  -h, --help            show this help message and exit
  --streamlines STREAMLINES
                        Input streamline file
  --envelope ENVELOPE   Input streamline envelope file
  --reference REFERENCE
                        Reference image used to automatically calulate binary
                        envelope if the envelope is not set. If reference is
                        not set, but start is set, start is used as reference
                        image for the envelope calculation.
  --start START         I





You can obtain such a tractogram with various software tools, e.g. MRtrix or MITK Diffusion. Since it is easy to use and fully automatic, we use TractSeg in our example.  
